<a href="https://colab.research.google.com/github/jh941213/news_summary_project/blob/main/DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.3 MB/s eta 0:00:00


In [ ]:
import pickle

# train_dataset을 pkl 파일에서 로드
with open("/content/drive/MyDrive/dacon/train_dataset.pkl", "rb") as f:
    train_dataset = pickle.load(f)

# test_dataset을 pkl 파일에서 로드
with open("/content/drive/MyDrive/dacon/test_dataset.pkl", "rb") as f:
    test_dataset = pickle.load(f)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset


model_name = "Laurie/news_classification_distilbert"
# 이전 체크포인트에서 모델 파라미터를 불러와서 적용
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8, ignore_mismatched_sizes=True)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Laurie/news_classification_distilbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([8]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=100,
)

# Trainer 객체 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=None  # validation 데이터셋 없이 학습
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,0.904600
200,0.503600
300,0.440200
400,0.369600
500,0.329900
600,0.313500
700,0.299300
800,0.296800
900,0.257600
1000,0.243600


TrainOutput(global_step=5928, training_loss=0.16917419914211498, metrics={'train_runtime': 8611.127, 'train_samples_per_second': 22.018, 'train_steps_per_second': 0.688, 'total_flos': 2.5117976288722944e+16, 'train_loss': 0.16917419914211498, 'epoch': 4.0})

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 테스트셋에 대한 예측
test_predictions = trainer.predict(test_dataset)
test_predictions = torch.argmax(torch.tensor(test_predictions.predictions), dim=1)


In [ ]:
# 제출 파일 생성
submission_df = pd.read_csv("/content/drive/MyDrive/dacon/sample_submission.csv")
submission_df["label"] = test_predictions
submission_df.to_csv("classi.csv", index=False)


In [ ]:
import shutil
shutil.move('/content/results/checkpoint-6500','/content/drive/MyDrive/dacon')

'/content/drive/MyDrive/dacon/checkpoint-6500'